## Predict housing prices from data patterns
in a *Kaggle* database with information from houses in Ames, Iowa containing:
* 80 descriptive variables
    * 37 numerical variables
    * 43 categorical variables
    
**Objective:** reduce prediction errors as much as possible using **XGBoost** models (Gradient Boosted Decision Trees)

**Author:** Leonardo Espin.

Licensing: This code is distributed under the Mozilla Public License Version 2.0. Date: 09/28/2018

In [35]:
# Loading libraries
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import cross_val_score,GridSearchCV
from sklearn import metrics
import matplotlib.pylab as plt
#set matplotlib options
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 4
pd.set_option('display.max_columns', 100)

## Data preparation
1. Load data

In [36]:
#Dataset file has to be downloaded from Kaggle after creating an account
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
print('Train: ' + str(train.shape) +' | Test: '+str(test.shape))
print('Training data fields: {}, total numerical fields: {}'
      .format(len(train.columns)-1,len(train.describe().columns)-1))
train.head()#shows the first few rows

Train: (1460, 81) | Test: (1459, 80)
Training data fields: 80, total numerical fields: 37


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


2. Comparing variables in train and test datasets

In [37]:
trainVars=set(train.columns)
testVars=set(test.columns)
print("\'SalePrice\' is the target variable ({} missing values)"
      .format(train.loc[:,'SalePrice'].isna().sum()))
print("Columns in train set and not in test :", trainVars-testVars) 
print("Columns in test set and not in train :", testVars-trainVars) 

'SalePrice' is the target variable (0 missing values)
Columns in train set and not in test : {'SalePrice'}
Columns in test set and not in train : set()


In [38]:
train['source']= 'train' #adding a source column first
test['source'] = 'test'
cols1=train.columns
test['SalePrice']=np.nan
data=pd.concat([train, test],ignore_index=True,sort=False)
cols2=data.columns
#data.drop(['Id'],axis=1,inplace=True)
print('Entire dataset : ' + str(data.shape)+'\n')
print(cols2==cols1) #check that original variable order has been preserved

Entire dataset : (2919, 82)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]


In [39]:
tmp=data.isna().sum()
tmp=tmp.loc[tmp >0] #variables with missing values
tmp.drop(index='SalePrice',inplace=True)

#concatenate number of missing values with variable type
dfA=pd.DataFrame(tmp,columns=['Missing'])
#dfB=pd.DataFrame(data.loc[:,list(tmp.index.values)].dtypes,columns=['Type'])
dfB=pd.DataFrame(data.loc[:,tmp.index].dtypes,columns=['Type'])
dfMissing=pd.concat([dfA,dfB],axis=1).sort_values('Missing',ascending=False)
#dfMissing

In [40]:
#Make list with variables with few missing rows (< 5 rows)
#these numerical variables will be imputed
fewNumMissing=list(dfA.loc[(dfA.loc[:,'Missing']<5) 
                           & (dfB.loc[:,'Type']=='float64')].index.values)#no int types above
#these variables have to be analyzed individually
fewCatMissing=list(dfA.loc[(dfA.loc[:,'Missing']<5) 
                           & (dfB.loc[:,'Type']=='object')].index.values)

#Make list with variables with too many missing rows (> 500 rows)
#these variables will be replaced with a binary variable which says if value is missing or not
manyCatMissing=list(dfMissing.loc[dfMissing.loc[:,'Missing']>500].index.values)

#Add LotFrontage,GarageYrBlt,MasVnrArea to numerical variables to be imputed
fewNumMissing=fewNumMissing +['LotFrontage','GarageYrBlt','MasVnrArea']

#Make list of remaining variables with missing data
otherMissing=list(set(dfMissing.index.values)
                  -set(fewNumMissing+fewCatMissing+manyCatMissing))
#dfMissing.loc[otherMissing]

In [41]:
for var in fewNumMissing:
    val=data[var].median()
    print('Filling \''+var+'\' with {}'.format(val) )
    data[var].fillna(data[var].median(),inplace=True)

Filling 'BsmtFinSF1' with 368.5
Filling 'BsmtFinSF2' with 0.0
Filling 'BsmtUnfSF' with 467.0
Filling 'TotalBsmtSF' with 989.5
Filling 'BsmtFullBath' with 0.0
Filling 'BsmtHalfBath' with 0.0
Filling 'GarageCars' with 2.0
Filling 'GarageArea' with 480.0
Filling 'LotFrontage' with 68.0
Filling 'GarageYrBlt' with 1979.0
Filling 'MasVnrArea' with 0.0


In [42]:
manyCatMissing.remove('FireplaceQu') #decided to keep categories for this variable
manyCatMissing

['PoolQC', 'MiscFeature', 'Alley', 'Fence']

In [43]:
for var in manyCatMissing: 
    data[var+'_missing']=data[var].apply(lambda x: 1 if pd.isnull(x) else 0)
    print(data[[var,var+'_missing']].head(10))
    data.drop(var,axis=1,inplace=True)

  PoolQC  PoolQC_missing
0    NaN               1
1    NaN               1
2    NaN               1
3    NaN               1
4    NaN               1
5    NaN               1
6    NaN               1
7    NaN               1
8    NaN               1
9    NaN               1
  MiscFeature  MiscFeature_missing
0         NaN                    1
1         NaN                    1
2         NaN                    1
3         NaN                    1
4         NaN                    1
5        Shed                    0
6         NaN                    1
7        Shed                    0
8         NaN                    1
9         NaN                    1
  Alley  Alley_missing
0   NaN              1
1   NaN              1
2   NaN              1
3   NaN              1
4   NaN              1
5   NaN              1
6   NaN              1
7   NaN              1
8   NaN              1
9   NaN              1
   Fence  Fence_missing
0    NaN              1
1    NaN              1
2    NaN       

In [44]:
var='FireplaceQu'
data[var].fillna('missing',inplace=True)
data[var].value_counts()

missing    1420
Gd          744
TA          592
Fa           74
Po           46
Ex           43
Name: FireplaceQu, dtype: int64

In [45]:
var='MSZoning'
data[var].fillna('missing',inplace=True)
var='Street'
data[var] = data[var].apply(lambda x: 1 if x == 'Pave' else 0)
var='LotShape'
data[var] = data[var].apply(lambda x: 1 if x == 'Reg' else 0)

RL         2265
RM          460
FV          139
RH           26
C (all)      25
missing       4
Name: MSZoning, dtype: int64

  * `Street`: make 'Pave' 1 else 0

  * `LotShape`: group IR2, IR3 and IR1, make binary, Reg = 1

In [48]:
keep=['LandContour','LotConfig','Neighborhood','Condition1','Exterior1st','Exterior2nd',
'MasVnrType','ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure',
'BsmtFinType1','HeatingQC','Electrical','KitchenQual','GarageType','GarageFinish',
'GarageQual','SaleType','SaleCondition']
for var in keep:
    data[var].fillna('missing',inplace=True)

Lvl    2622
HLS     120
Bnk     117
Low      60
Name: LandContour, dtype: int64
Inside     2133
Corner      511
CulDSac     176
FR2          85
FR3          14
Name: LotConfig, dtype: int64
NAmes      443
CollgCr    267
OldTown    239
Edwards    194
Somerst    182
NridgHt    166
Gilbert    165
Sawyer     151
NWAmes     131
SawyerW    125
Mitchel    114
BrkSide    108
Crawfor    103
IDOTRR      93
Timber      72
NoRidge     71
StoneBr     51
SWISU       48
ClearCr     44
MeadowV     37
BrDale      30
Blmngtn     28
Veenker     24
NPkVill     23
Blueste     10
Name: Neighborhood, dtype: int64
Norm      2511
Feedr      164
Artery      92
RRAn        50
PosN        39
RRAe        28
PosA        20
RRNn         9
RRNe         6
Name: Condition1, dtype: int64
VinylSd    1025
MetalSd     450
HdBoard     442
Wd Sdng     411
Plywood     221
CemntBd     126
BrkFace      87
WdShing      56
AsbShng      44
Stucco       43
BrkComm       6
Stone         2
AsphShn       2
CBlock        2
missing     

In [49]:
var='Utilities'
data[var] = data[var].apply(lambda x: 1 if x == 'AllPub' else 0)
var='LandSlope'
data[var] = data[var].apply(lambda x: 1 if x == 'Gtl' else 0)
var='Condition2'
data.drop([var],axis=1,inplace=True)
var='BldgType'
data[var] = data[var].apply(lambda x: 1 if x in ['1Fam','TwnhsE'] else 0)
var='HouseStyle'
data[var] = data[var].apply(lambda x: 'others' if x not in ['1Story','2Story'] else x)
var='RoofStyle'
data[var] = data[var].apply(lambda x: x if x in ['Gable','Hip'] else 'others')
var='RoofMatl'
data.drop([var],axis=1,inplace=True)
var='BsmtFinType2'
data[var] = data[var].apply(lambda x: 1 if x == 'Unf' else 0)
var='Heating'
data[var] = data[var].apply(lambda x: 1 if x == 'GasA' else 0)
var='CentralAir'
data[var] = data[var].apply(lambda x: 1 if x == 'Y' else 0)
var='Functional'
data[var] = data[var].apply(lambda x: 1 if x == 'Typ' else 0)
var='GarageCond'
data[var] = data[var].apply(lambda x: 1 if x == 'TA' else 0)
var='PavedDrive'
data[var] = data[var].apply(lambda x: 1 if x == 'Y' else 0)

1    2916
0       3
Name: Utilities, dtype: int64

In [63]:
cat_vars = list(data.select_dtypes(['object']))
cat_vars.remove('source')
data = pd.get_dummies(data, columns=cat_vars)
#data.head(10)

,Id,MSSubClass,LotFrontage,LotArea,Street,LotShape,Utilities,LandSlope,BldgType,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,CentralAir,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Functional,Fireplaces,GarageYrBlt,GarageCars,GarageArea,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,source,...,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,Electrical_missing,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,KitchenQual_missing,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_Po,FireplaceQu_TA,FireplaceQu_missing,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_missing,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageFinish_missing,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageQual_missing,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleType_missing,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,1,1,1,1,1,7,5,2003,2003,196.0,706.0,1,0.0,150.0,856.0,1,1,856,854,0,1710,1.0,0.0,2,1,3,1,8,1,0,2003.0,2.0,548.0,1,1,0,61,0,0,0,0,0,2,2008,208500.0,train,...,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
1,2,20,80.0,9600,1,1,1,1,1,6,8,1976,1976,0.0,978.0,1,0.0,284.0,1262.0,1,1,1262,0,0,1262,0.0,1.0,2,0,3,1,6,1,1,1976.0,2.0,460.0,1,1,298,0,0,0,0,0,0,5,2007,181500.0,train,...,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
2,3,60,68.0,11250,1,0,1,1,1,7,5,2001,2002,162.0,486.0,1,0.0,434.0,920.0,1,1,920,866,0,1786,1.0,0.0,2,1,3,1,6,1,1,2001.0,2.0,608.0,1,1,0,42,0,0,0,0,0,9,2008,223500.0,train,...,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
3,4,70,60.0,9550,1,0,1,1,1,7,5,1915,1970,0.0,216.0,1,0.0,540.0,756.0,1,1,961,756,0,1717,1.0,0.0,1,0,3,1,7,1,1,1998.0,3.0,642.0,1,1,0,35,272,0,0,0,0,2,2006,140000.0,train,...,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
4,5,60,84.0,14260,1,0,1,1,1,8,5,2000,2000,350.0,655.0,1,0.0,490.0,1145.0,1,1,1145,1053,0,2198,1.0,0.0,2,1,4,1,9,1,1,2000.0,3.0,836.0,1,1,192,84,0,0,0,0,0,12,2008,250000.0,train,...,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
5,6,50,85.0,14115,1,0,1,1,1,5,5,1993,1995,0.0,732.0,1,0.0,64.0,796.0,1,1,796,566,0,1362,1.0,0.0,1,1,1,1,5,1,0,1993.0,2.0,480.0,1,1,40,30,0,320,0,0,700,10,2009,143000.0,train,...,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
6,7,20,75.0,10084,1,1,1,1,1,8,5,2004,2005,186.0,1369.0,1,0.0,317.0,1686.0,1,1,1694,0,0,1694,1.0,0.0,2,0,3,1,7,1,1,2004.0,2.0,636.0,1,1,255,57,0,0,0,0,0,8,2007,307000.0,train,...,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
7,8,60,68.0,10382,1,0,1,1,1,7,6,1973,1973,240.0,859.0,0,32.0,216.0,1107.0,1,1,1107,983,0,2090,1.0,0.0,2,1,3,1,7,1,2,1973.0,2.0,484.0,1,1,235,204,228,0,0,0,350,11,2009,200000.0,train,...,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
8,9,50,51.0,6120,1,1,1,1,1,7,5,1931,1950,0.0,0.0,1,0.0,952.0,952.0,1,1,1022,752,0,1774,0.0,0.0,2,0,2,2,8,0,2,1931.0,2.0,468.0,1,1,90,0,205,0,0,0,0,4,2008,129900.0,train,...,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
9,10,190,50.0,7420,1,1,1,1,0,5,6,1939,1950,0.0,851.0,1,0.0,140.0,991.0,1,1,1077,0,0,1077,1.0,0.0,1,0,2,2,5,1,2,1939.0,1.0,205.0,1,1,0,4,0,0,0,0,0

12. Separate train and test data and store in new files

In [64]:
train = data.loc[data['source']=='train'].copy() #copy is to make sure we're not chain-modifying data
test = data.loc[data['source']=='test'].copy()

In [65]:
train.drop('source',axis=1,inplace=True)
test.drop(['source','SalePrice'],axis=1,inplace=True)

  * Set target and predictor variables

In [66]:
target='SalePrice'
predictors =list(train.columns)
predictors.remove(target)
predictors.remove('Id')

  * set regressor

In [ ]:
xgb4 = XGBRegressor( #classifier because of binary targer variable
        learning_rate =0.01, #default value (high)
        n_estimators=3593,  #OPTIMIZED 
        max_depth=3,        #OPTIMIZED 
        min_child_weight=2, #OPTIMIZED
        gamma=0,            #OPTIMIZED 
        subsample=0.5,      #OPTIMIZED 
        colsample_bytree=0.3,#OPTIMIZED 
        nthread=4,
        scale_pos_weight=1, #default
        random_state=27,    #Random number seed
        reg_alpha=530,      #OPTIMIZED 
        reg_lambda=1        #OPTIMIZED 
)    

xgb4.fit(train[predictors],train[target])
test_preds = xgb4.predict(test[predictors])

# The lines below shows you how to save your data in the format needed to score it in the competition
output = pd.DataFrame({'Id': test.Id,
                       'SalePrice': test_preds})

output.to_csv('submission.csv', index=False)